# Prototype Visualization for MProtoNet3D - Google Colab

This notebook visualizes the learned prototypes from your trained brain tumor segmentation model.

**Steps:**
1. Mount Google Drive (where your model is saved)
2. Clone GitHub repository
3. Download preprocessed data from S3
4. Run prototype visualization
5. Save visualizations to Google Drive

## Step 1: Check GPU

In [ ]:
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Enable memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

## Step 2: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set path to your trained model in Google Drive
MODEL_PATH = '/content/drive/MyDrive/brain_tumor_checkpoints/best_model_YYYYMMDD-HHMMSS.keras'
OUTPUT_DIR = '/content/drive/MyDrive/brain_tumor_checkpoints/prototype_visualizations'

print(f"Model will be loaded from: {MODEL_PATH}")
print(f"Visualizations will be saved to: {OUTPUT_DIR}")
print("\n⚠️ Update MODEL_PATH above with your actual model filename!")

## Step 3: Install Dependencies

In [ ]:
%%capture
!pip install h5py numpy tensorflow keras matplotlib awscli boto3 tqdm -q

## Step 4: Clone GitHub Repository

In [ ]:
import os

GITHUB_REPO_URL = "https://github.com/dariamarc/brainTumorSurvival.git"
repo_name = GITHUB_REPO_URL.split('/')[-1].replace('.git', '')

print(f"Cloning repository: {GITHUB_REPO_URL}")
print("-" * 60)

# Change to /content first
os.chdir('/content')

# Remove if exists
if os.path.exists(f'/content/{repo_name}'):
    !rm -rf /content/{repo_name}
    print(f"Removed existing directory: {repo_name}")

# Clone
!git clone {GITHUB_REPO_URL}

# Change to repo directory
os.chdir(f'/content/{repo_name}')
print(f"\n✓ Changed to directory: {os.getcwd()}")

# List files
print("\nRepository contents:")
!ls -la

## Step 5: Configure AWS Credentials

In [ ]:
from google.colab import userdata
import os

print("Configuring AWS credentials...")
print("-" * 60)

try:
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
    print("✓ AWS credentials loaded from Colab Secrets")
except Exception as e:
    print("✗ Error loading AWS credentials")
    print(f"Error: {e}")
    print("\nPlease add AWS credentials to Colab Secrets (🔑 icon)")
    raise

## Step 6: Download Preprocessed Data from S3

In [ ]:
# ============================================================================
# UPDATE WITH YOUR S3 DETAILS
# ============================================================================
S3_BUCKET = 'your-brats2020-data'
S3_PATH = 'preprocessed_data'
AWS_REGION = 'eu-central-1'
# ============================================================================

LOCAL_DATA_PATH = '/content/brainTumorData_preprocessed'

print("="*60)
print("DOWNLOADING PREPROCESSED DATA FROM S3")
print("="*60)

os.environ['AWS_DEFAULT_REGION'] = AWS_REGION

print(f"Source: s3://{S3_BUCKET}/{S3_PATH}")
print(f"Destination: {LOCAL_DATA_PATH}")
print("-" * 60)

!mkdir -p {LOCAL_DATA_PATH}
!aws s3 sync s3://{S3_BUCKET}/{S3_PATH} {LOCAL_DATA_PATH}

# Verify
if os.path.exists(LOCAL_DATA_PATH):
    file_count = sum([len(files) for r, d, files in os.walk(LOCAL_DATA_PATH)])
    print(f"\n✓ Downloaded {file_count:,} files")
    print(f"✓ Data ready at: {LOCAL_DATA_PATH}")
else:
    print("\n✗ Download failed!")
    raise FileNotFoundError(f"Data not found at {LOCAL_DATA_PATH}")

## Step 7: Import Modules

In [ ]:
import sys

# Ensure repository is in Python path
repo_dir = f'/content/{repo_name}'
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

# Import modules
from model import MProtoNet3D_Segmentation_Keras
from losses import CombinedLoss
from data_generator import MRIDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm

print("✓ All modules imported successfully!")

## Step 8: Load Trained Model

In [ ]:
print(f"Loading model from: {MODEL_PATH}")
print("-" * 60)

model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={'CombinedLoss': CombinedLoss}
)

print("✓ Model loaded successfully!")
print(f"\nModel summary:")
print(f"  - Number of prototypes: {model.num_prototypes}")
print(f"  - Number of classes: {model.num_classes}")
print(f"  - Prototypes per class: {model.num_prototypes_per_class}")
print(f"  - Prototype shape: {model.prototype_shape_tuple}")

## Step 9: Extract Prototype Vectors

In [ ]:
prototype_vectors = model.prototype_vectors.numpy()

print("Prototype Information:")
print("="*60)
print(f"Prototype vectors shape: {prototype_vectors.shape}")
print(f"Number of prototypes: {prototype_vectors.shape[0]}")
print(f"Feature dimensions: {prototype_vectors.shape[1]}")

# Show class assignments
print(f"\nPrototype class assignments:")
class_names = ['GD-enhancing tumor (ET)', 'Peritumoral edema (ED)', 'Necrotic core (NCR/NET)']
for i in range(model.num_prototypes):
    class_idx = i // model.num_prototypes_per_class
    print(f"  Prototype {i:2d} -> Class {class_idx} ({class_names[class_idx]})")

## Step 10: Create Data Generator

In [ ]:
print("Creating data generator...")

data_generator = MRIDataGenerator(
    LOCAL_DATA_PATH,
    batch_size=1,
    num_slices=96,
    num_volumes=369,
    split_ratio=0.2,
    subset='train',
    shuffle=False,
    random_state=42
)

print(f"✓ Data generator created")
print(f"  - Training volumes: {len(data_generator.indices)}")
print(f"  - Batches: {len(data_generator)}")

## Step 11: Find Closest Patches to Each Prototype

This searches through training data to find image patches that most strongly activate each prototype.

In [ ]:
NUM_SAMPLES_TO_CHECK = 50  # Adjust based on how thorough you want the search

print(f"Searching through {NUM_SAMPLES_TO_CHECK} volumes for closest prototype matches...")
print("This may take 5-10 minutes...")
print("-" * 60)

# Create feature extractor
feature_extractor = tf.keras.Model(
    inputs=model.input,
    outputs=model.add_ons.output
)

num_prototypes = prototype_vectors.shape[0]

# Track best matches for each prototype
best_matches = {i: {
    'activation': -np.inf,
    'volume_idx': None,
    'slice_idx': None,
    'location': None,
    'image_patch': None,
    'mask_patch': None
} for i in range(num_prototypes)}

# Iterate through data
for sample_idx in tqdm(range(min(NUM_SAMPLES_TO_CHECK, len(data_generator)))):
    batch_x, batch_y = data_generator[sample_idx]
    
    # Extract features
    features = feature_extractor.predict(batch_x, verbose=0)
    
    # Compute prototype activations
    if model.f_dist == 'l2':
        distances = model.l2_convolution_3D(features).numpy()
        similarities = model.distance_2_similarity(distances).numpy()
    else:
        similarities = model.cosine_convolution_3D(features).numpy()
    
    # For each prototype, check if this is the best activation
    for proto_idx in range(num_prototypes):
        proto_similarities = similarities[0, :, :, :, proto_idx]
        max_activation = np.max(proto_similarities)
        
        if max_activation > best_matches[proto_idx]['activation']:
            max_loc = np.unravel_index(np.argmax(proto_similarities), proto_similarities.shape)
            d, h, w = max_loc
            
            # Extract patch
            patch_size = 16
            h_start = max(0, h - patch_size // 2)
            h_end = min(batch_x.shape[2], h + patch_size // 2)
            w_start = max(0, w - patch_size // 2)
            w_end = min(batch_x.shape[3], w + patch_size // 2)
            
            image_patch = batch_x[0, d, h_start:h_end, w_start:w_end, :]
            if len(batch_y.shape) == 5:
                mask_patch = batch_y[0, d, h_start:h_end, w_start:w_end, :]
            else:
                mask_patch = batch_y[0, d, h_start:h_end, w_start:w_end]
            
            best_matches[proto_idx] = {
                'activation': max_activation,
                'volume_idx': data_generator.volume_ids[data_generator.indices[sample_idx]],
                'slice_idx': d,
                'location': (h, w),
                'image_patch': image_patch,
                'mask_patch': mask_patch
            }

print("\n✓ Search complete!")
print(f"Found best matches for all {num_prototypes} prototypes")

## Step 12: Visualize Prototypes by Class

In [ ]:
!mkdir -p {OUTPUT_DIR}

modality_names = ['T1', 'T1ce', 'T2', 'FLAIR']

for class_idx in range(model.num_classes):
    print(f"Visualizing prototypes for class {class_idx}: {class_names[class_idx]}")
    
    start_proto = class_idx * model.num_prototypes_per_class
    end_proto = start_proto + model.num_prototypes_per_class
    
    fig, axes = plt.subplots(model.num_prototypes_per_class, 5, 
                             figsize=(25, 5 * model.num_prototypes_per_class))
    
    for local_idx, proto_idx in enumerate(range(start_proto, end_proto)):
        match = best_matches[proto_idx]
        
        # Column 0: Info
        axes[local_idx, 0].text(0.5, 0.5,
            f"Prototype {proto_idx}\n"
            f"Class: {class_idx}\n"
            f"Activation: {match['activation']:.4f}\n"
            f"Volume: {match['volume_idx']}\n"
            f"Slice: {match['slice_idx']}\n"
            f"Location: {match['location']}",
            ha='center', va='center', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        axes[local_idx, 0].axis('off')
        
        # Columns 1-4: Modalities
        if match['image_patch'] is not None:
            for mod_idx in range(4):
                axes[local_idx, mod_idx + 1].imshow(match['image_patch'][:, :, mod_idx], cmap='gray')
                axes[local_idx, mod_idx + 1].set_title(f"{modality_names[mod_idx]}", fontsize=10)
                axes[local_idx, mod_idx + 1].axis('off')
    
    plt.suptitle(f"Learned Prototypes for Class {class_idx}: {class_names[class_idx]}",
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/prototypes_class_{class_idx}.png", dpi=300, bbox_inches='tight')
    print(f"  ✓ Saved: prototypes_class_{class_idx}.png")
    plt.show()
    plt.close()

## Step 13: Visualize Activation Maps with Masks

In [ ]:
for class_idx in range(model.num_classes):
    start_proto = class_idx * model.num_prototypes_per_class
    end_proto = start_proto + model.num_prototypes_per_class
    
    fig, axes = plt.subplots(2, model.num_prototypes_per_class, 
                             figsize=(model.num_prototypes_per_class * 4, 8))
    
    for local_idx, proto_idx in enumerate(range(start_proto, end_proto)):
        match = best_matches[proto_idx]
        
        if match['image_patch'] is not None:
            # Show T1ce image
            axes[0, local_idx].imshow(match['image_patch'][:, :, 1], cmap='gray')
            axes[0, local_idx].set_title(f"Proto {proto_idx}\nT1ce", fontsize=10)
            axes[0, local_idx].axis('off')
            
            # Show with mask overlay
            if len(match['mask_patch'].shape) == 3:
                mask_vis = np.argmax(match['mask_patch'], axis=-1)
            else:
                mask_vis = match['mask_patch']
            
            axes[1, local_idx].imshow(match['image_patch'][:, :, 1], cmap='gray')
            axes[1, local_idx].imshow(mask_vis, cmap='jet', alpha=0.5)
            axes[1, local_idx].set_title(f"With Mask\nAct: {match['activation']:.3f}", fontsize=10)
            axes[1, local_idx].axis('off')
    
    plt.suptitle(f"Prototype Activation Maps - {class_names[class_idx]}",
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_DIR}/activation_maps_class_{class_idx}.png", dpi=300, bbox_inches='tight')
    print(f"✓ Saved: activation_maps_class_{class_idx}.png")
    plt.show()
    plt.close()

## Step 14: Visualize Prototype Statistics

In [ ]:
# Compute statistics
proto_norms = np.linalg.norm(prototype_vectors.reshape(num_prototypes, -1), axis=1)
proto_flat = prototype_vectors.reshape(num_prototypes, -1)
similarities = np.dot(proto_flat, proto_flat.T) / (
    np.outer(proto_norms, proto_norms) + 1e-8
)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Norms
axes[0].bar(range(num_prototypes), proto_norms)
axes[0].set_xlabel('Prototype Index')
axes[0].set_ylabel('L2 Norm')
axes[0].set_title('Prototype Vector Magnitudes')
axes[0].grid(True, alpha=0.3)

# Plot 2: Activations
activations = [best_matches[i]['activation'] for i in range(num_prototypes)]
axes[1].bar(range(num_prototypes), activations)
axes[1].set_xlabel('Prototype Index')
axes[1].set_ylabel('Max Activation')
axes[1].set_title('Best Activation Strength per Prototype')
axes[1].grid(True, alpha=0.3)

# Plot 3: Similarity matrix
im = axes[2].imshow(similarities, cmap='coolwarm', vmin=-1, vmax=1)
axes[2].set_xlabel('Prototype Index')
axes[2].set_ylabel('Prototype Index')
axes[2].set_title('Prototype Similarity Matrix')
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.savefig(f"{OUTPUT_DIR}/prototype_statistics.png", dpi=300, bbox_inches='tight')
print("✓ Saved: prototype_statistics.png")
plt.show()
plt.close()

## Step 15: Summary

In [ ]:
print("="*70)
print("✓ PROTOTYPE VISUALIZATION COMPLETE!")
print("="*70)
print(f"\nAll visualizations saved to Google Drive:")
print(f"  {OUTPUT_DIR}/")
print(f"\nFiles created:")
print(f"  - prototypes_class_0.png (GD-enhancing tumor)")
print(f"  - prototypes_class_1.png (Peritumoral edema)")
print(f"  - prototypes_class_2.png (Necrotic core)")
print(f"  - activation_maps_class_*.png (Activation heatmaps)")
print(f"  - prototype_statistics.png (Statistics and similarities)")
print("="*70)

# List files
!ls -lh {OUTPUT_DIR}